In [14]:
using Revise

# <center>Introducción a</center>
<center><a href="http://julialang.org/" target="_blank"><img src="figs/julia_logo_small.png"/></a>



### <center>Daniel Molina Cabrera</center> <br><center><a href="mailto:dmolina@decsai.ugr.es">dmolina@decsai.ugr.es</a></center>

# ¿Qué es Julia?

# Motivación de Julia en 2009

> We want a language that’s **open source**, with a liberal license. We want **the speed of C** with the **dynamism of Ruby**. We want a language that’s homoiconic, with true macros like Lisp, but with obvious, **familiar mathematical notation like Matlab**. We want something as usable for **general programming as Python**, **as  easy for statistics as R**, as natural for string processing as Perl, as **powerful for linear algebra as Matlab**, as good at gluing programs together as the shell. Something that is dirt simple to learn, yet keeps the most serious hackers happy. We want it **interactive** and we want **it compiled**.

> (Did we mention it should be as fast as C?)

# Por tanto

- Es un Lenguaje Software Libre. 
- De propósito general, pero hecho por y para científicos. 
- Eficiente (no interpreta las funciones, las compila y ejecuta).
- Muy similar a Python.


# Código en Python _vs_ Julia
```python
def fib(n):
    if n <= 1:
        return 1
    else:
        return fib(n-1)+fib(n-2)
    
%time fib(40)```



<center><img src="figs/julia_nature.png"></center>

# Evolución de Julia

# Origen de Julia

- Desarrollado por varios estudiantes de Doctorado del MIT desde 2009, primera versión pública en 2012.
- Versión 1.0 en Agosto de 2018.
    - Más de 2 millones de descargas, estimado medio millón de usuarios habituales.
    - 750 han subido commit (yo incluído).
    - +2400 paquetes, algunos de mucha calidad.

# Recientes Hitos de Julia

- Sistema de Paquetes a Final de 2018.
- Reescrito depurador en 2019.

# Futuros cambios

- Versión 1.3 con mejor soporte de hebras (Release).
- Mejorar la carga de librerías en Versión 1.4.
- Permitir crear ejecutables.

# Motivo de Julia


## Evitar el problema del doble lenguaje

- Uno sencillo para flexibilidad, pero lento. 
- Uno complejo pero rápido.

<center> <img src="figs/two2.png"></center>

# No tiene por qué ser asi

### <center>Algunos Benchmarks</center>

<center><img src="figs/benchmarks.png" width="80%"/></center>

<center><img src="figs/julia_vs_others.png"/></center>

# El objetivo es simplificar la vida

<center> <img src="figs/two1.png"></center>

<center> <img src="figs/two3.png"></center>

# Mi caso es parecido

Trabajo en Metaheurísticas, la mayoría usa Matlab (yo Python), en problemas complejos

## Ejecutando código externo

- Pasado en Matlab, problemas de licencia.
- Ejecutar en Octave supone tardar mucho más. 
- Pasarlo a Python es conflictivo:
    - Sintaxis distinta.
    - Posibles problemas: En Matlab índices desde 1, en Python por 0. 
    - En Python es necesario usar numpy, sintaxis más alejada de Matlab.

### Proceso de mis algoritmos

- Python para prototipado.
- Numpy para rendimiento.
- Evitar condiciones, usar operaciones vectorizadas.
- Identificar cuellos de botella.
- A veces C++ para reimplementar esas partes (Cython).


### Usando Julia

- No es necesario Numpy ni cython.
- Librerías sólo disponibles en Python usando PyCall.
- Más fácil llamar código C/C++ que desde Python.

# Julia vs Python

## Semejanzas

- Sintaxis muy similar.
- Entorno Interactivo (REPL) como IPython.

<center><img src="figs/repl.png"></center>


- Modo Julia, por defecto.
- Modo Package \[, para buscar y gestionar paquetes.
- Modo Shell \; para ejecutar comandos.

Vamos a verlo.

# Repositorio Oficial de Paquetes

In [38]:
using Pkg
Pkg.add("OhMyREPL")

 Resolving package versions...
  Updating `~/.julia/environments/v1.2/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.2/Manifest.toml`
 [no changes]


In [40]:
using OhMyREPL

# Más semejanzas con Python

- Uso de paquetes, importa con **import** o con **using**. 
- Sistema de tests automáticos.

In [43]:
import Distributions
rand(Distributions.Uniform(-5, 5), 3)

3-element Array{Float64,1}:
 -4.039227074993034 
  2.5703006007464797
  1.5694341722914658

In [44]:
using Distributions: Uniform 
rand(Uniform(-5, 5), 3)

3-element Array{Float64,1}:
 0.35264769161857323
 1.179009962504236  
 2.0017282827132865 

In [46]:
using Distributions
rand(Uniform(-5, 5), 3)

3-element Array{Float64,1}:
 -4.753048394464175 
 -2.809094029085335 
 -2.9108722364243977

# Diferencias en la sintaxis

- _def_ => _function_ (como Matlab).
- No usa tabulador para distinguir, usa **end**. 
- Lo bueno es que no tiene begin, y el _end_ es reducido (no antes de un else, por ejemplo).

In [59]:
function fibo(n::Int)
    (a, b) = (1, 1)
    
    for _ in 1:n-1
        (a, b) = (a+b, a)   
        end
    a
end
@time @show fibo(40)

fibo(40) = 165580141
  0.010907 seconds (20.29 k allocations: 1.106 MiB)


165580141

# Tipos opcionales
- No es necesario, él deduce cuando se llama al método.
- Existen tipos genéricos, útiles para evitar errores (Int, Float, Real, Number, AbstractString, ....

In [67]:
sphere(x::AbstractArray)= sum(x.^2)
sphere(3)

MethodError: MethodError: no method matching sphere(::Int64)
Closest candidates are:
  sphere(!Matched::AbstractArray) at In[67]:1

In [70]:
sphere([3, 4, 5])

50

# Es eficiente porque se compila según el tipo concreto

In [68]:
@code_warntype sphere([3, 4, 5])

Variables
  #self#::Core.Compiler.Const(sphere, false)
  x::Array{Int64,1}

Body::Int64
1 ─ %1 = Core.apply_type(Base.Val, 2)::Core.Compiler.Const(Val{2}, false)
│   %2 = (%1)()::Core.Compiler.Const(Val{2}(), false)
│   %3 = Base.broadcasted(Base.literal_pow, Main.:^, x, %2)::Base.Broadcast.Broadcasted{Base.Broadcast.DefaultArrayStyle{1},Nothing,typeof(Base.literal_pow),Tuple{Base.RefValue{typeof(^)},Array{Int64,1},Base.RefValue{Val{2}}}}
│   %4 = Base.materialize(%3)::Array{Int64,1}
│   %5 = Main.sum(%4)::Int64
└──      return %5


# Es eficiente porque se compila según el tipo concreto

In [77]:
fabsurda(x)=x^2+3;  

In [78]:
@code_warntype fabsurda(3)

Variables
  #self#::Core.Compiler.Const(fabsurda, false)
  x::Int64

Body::Int64
1 ─ %1 = Core.apply_type(Base.Val, 2)::Core.Compiler.Const(Val{2}, false)
│   %2 = (%1)()::Core.Compiler.Const(Val{2}(), false)
│   %3 = Base.literal_pow(Main.:^, x, %2)::Int64
│   %4 = (%3 + 3)::Int64
└──      return %4


In [80]:
@code_warntype fabsurda(3.3)

Variables
  #self#::Core.Compiler.Const(fabsurda, false)
  x::Float64

Body::Float64
1 ─ %1 = Core.apply_type(Base.Val, 2)::Core.Compiler.Const(Val{2}, false)
│   %2 = (%1)()::Core.Compiler.Const(Val{2}(), false)
│   %3 = Base.literal_pow(Main.:^, x, %2)::Float64
│   %4 = (%3 + 3)::Float64
└──      return %4


In [84]:
@code_llvm fabsurda(3)


;  @ In[77]:1 within `fabsurda'
define i64 @julia_fabsurda_17527(i64) {
top:
; ┌ @ intfuncs.jl:244 within `literal_pow'
; │┌ @ int.jl:54 within `*'
    %1 = mul i64 %0, %0
; └└
; ┌ @ int.jl:53 within `+'
   %2 = add i64 %1, 3
; └
  ret i64 %2
}


# Diferente inicio del índice
- Empieza en 1, no en 0. 

No es tan problemático, se suele usar iterador y/o enumerate.

In [95]:
values = ["Uno", "Dos", "Tres"]

for (i, val) in enumerate(values)
    println("$i: $val")
end

1: Uno
2: Dos
3: Tres


# Estructuras de Datos en el lenguaje

- Arrays, dentro del propio lenguaje.
- String, UTF-8.
- Diccionarios.
- Conjuntos.

In [108]:
dict = Dict(i=>val for (i, val) in enumerate(values))

Dict{Int64,String} with 3 entries:
  2 => "Dos"
  3 => "Tres"
  1 => "Uno"

In [100]:
println(keys(dict))

[2, 3, 1]


In [107]:
collect(Set([1, 2, 4, 2, 4]))

3-element Array{Int64,1}:
 4
 2
 1

# Modo de Ayuda

Uso de cadenas descriptivas para cada función. 

In [102]:
"""
Esta función permite calculo el número de fibonacci.

n: Entero del cual calcular fibonacci.
"""
function fibo(n::Int)
    (a, b) = (1, 1)
    
    for _ in 1:n-1
        (a, b) = (a+b, a)
    end
    a
end
@time @show fibo(40)

fibo(40) = 165580141
  0.011626 seconds (20.29 k allocations: 1.106 MiB)


165580141

In [104]:
?fibo

search: fibo fib2 fib failprob unsafe_pointer_to_objref



Esta función permite calculo el número de fibonacci.

n: Entero del cual calcular fibonacci.


Se puede construir fácilmente la documentación, Paquete [Documenter.jl](https://juliadocs.github.io/Documenter.jl/stable/):
- Usa Markdown para describirlo para los usuarios.
- Permite añadir la documentación del API. 
- Usar ejemplos que sirven también de test. 

# Comunidad Científica

Librerías/Paquetes populares en: [https://pkg.julialang.org/docs/](https://pkg.julialang.org/docs/)

- Notebook: [IJulia](https://pkg.julialang.org/docs/IJulia/nfu7T/1.20.2/)
- Científico: [QueryVerse](https://www.queryverse.org/)
    - Librería DataFrames: [DataFrames.jl](https://github.com/JuliaData/DataFrames.jl)
    - Visualización: [VegaLite](https://www.queryverse.org/VegaLite.jl/stable/)
- Visualización: [Gadfly](https://gadflyjl.org/stable/tutorial/), [Plots](https://docs.juliaplots.org/latest/tutorial/#tutorial-1), [StatsPlots](https://github.com/JuliaPlots/StatsPlots.jl)
- Librerías de Deep Learning: [Flux.jl](https://fluxml.ai/), [KNet.jl](https://github.com/denizyuret/Knet.jl).


# Comunidad científica

## Librerías del Estado del Arte
- Librería de optimización: [JuMP](https://pkg.julialang.org/docs/JuMP/DmXqY/0.20.1/quickstart.html#Quick-Start-Guide-1)
- Ecuaciones Diferenciales: [DifferentialEquations](https://docs.juliadiffeq.org/latest/)
- Librerías estadística: [Distributions](https://juliastats.org/Distributions.jl/latest/starting/)

## Más genéricas
- Páginas web: [Genie](https://genieframework.github.io/Genie.jl/)
- Base de Datos: [Octo](https://github.com/wookay/Octo.jl)

# Benchmarks

# Ejemplo sencillo

In [29]:
function smallestdivisall(n::Int64)
    for i = 1:factorial(n)
        for j = 1:n
            if i % j !=0  
                break
            elseif j == n
                return i
            end
        end
    end
end


smallestdivisall (generic function with 1 method)

# Resultados

<center><img src="figs/jit-comparison.png"/></center>

Fuente: https://randyzwitch.com/python-pypy-julia-r-pqr-jit-just-in-time-compiler/


### Benchmarks de funciones comunes

Rápidamente: Los tiempos de Julia son semejantes a los de C, Matlab/Python/R son órdenes de magnitud más lentos 
<center><img src="figs/benchmarks.png"/></center>

# Carga de Datos

<center> <img width="75%" src="figs/carga_datos.png"></center>

# Origen de las librerías

<center><img src="figs/two_language_problem.jpg"/></center>